## Collection and Preparation:

In a MLOps approch, this jupyter notebook will Collect, Create the dataset and store it into a AWS like Bucket ( MiniIo).

Import and create the folder data:

In [25]:
! pip install pandas
! pip install tqdm
! pip install numpy
! pip install boto3
! mkdir data


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
mkdir: cannot create directory ‘data’: File exists


In [26]:
import os
import pandas as pd
import tqdm
import numpy as np
import random
import boto3
from botocore.exceptions import ClientError


In Montréal, in 2025, we have 15 skate spots. Some skatepark are greater to skate than others, for example, Ahuntsic 's skatepark have more modules than Verdun 's skate park.

Why create a model if you already know which parks is the best?
Depending on the traffic, on the day (and the occupation, because some skateparks are more busy the week end than others) the distance, we don't go to the same spot every session.

## The problem:
Ususally my friends ask me at the last minute: "Seb, where we go today?" And every time, I have no idea of which skatepark could be the best option. So that's a great problem to solve as a MLOps Software Engineer like me :) 


In [27]:
skateparks_list = [
    "Ahuntsic"         , # 0
    "VanHorne"         , # 1
    "Verdun"           , # 2
    "Lasalle"          , # 3
    "Préfontaine"      , # 4
    "Boucherville"     , # 5
    "Taz"              , # 6
    "Spin"             , # 7
    "Saint Jérome"     , # 8
    "Saint Sauveur"    , # 9
    "Assomption"       , # 10
    "Benny"            , # 11
    "Dorval"           , # 12
    "Magog"            , # 13
    "Berthierville"      # 14
]

# Weather: 
# Rain or Snow  = 0
# Clouds + Wind = 1
# Clouds        = 2
# Blue sky      = 3

weather_dict = {
    0: "Rain or Snow",
    1: "Windy or heavily cloudy",
    2: "cloudy",
    3: "Blue sky"
}

Temperature will be in °C
traffic time: in minutes

We will give numbers index for the weekday:
0: Monday -> 6: Sunday

#### For this test, we will the index of the skatepark, the weather, the temperature and the traffic time

In [28]:
N=  20
data = []

for idx in tqdm.tqdm(range(N)):
    try:
        weather = random.randint(0, 3)
        temperature = np.clip(np.random.normal(22,6), -5, 38)
        traffic_time = np.clip(np.random.normal(25, 10), 5, 150)

        print(f"\n\n météo: {weather_dict[weather]}, a {temperature}°C\t, temps de traffic: {traffic_time}min")
        for idx_spot, spot_name in enumerate(skateparks_list):
            satisfaction = input(f"{spot_name}\tSatisfaction?")
            data.append({
                "spot_name": spot_name,
                "index spot": idx_spot,
                "weather": weather,
                "temperature": round(float(temperature), 1),
                "traffic_time": round(float(traffic_time), 1),
                "satisfaction": round(float(satisfaction), 2)
            })
    except Exception as e:
        print(e)


  0%|          | 0/20 [00:00<?, ?it/s]



 météo: cloudy, a 28.586508525969933°C	, temps de traffic: 23.727882414478277min


Ahuntsic	Satisfaction? 6
VanHorne	Satisfaction? 5
Verdun	Satisfaction? 3
Lasalle	Satisfaction? 6
Préfontaine	Satisfaction? 5
Boucherville	Satisfaction? 5
Taz	Satisfaction? 1
Spin	Satisfaction? 1
Saint Jérome	Satisfaction? 7
Saint Sauveur	Satisfaction? 8
Assomption	Satisfaction? 8
Benny	Satisfaction? 6
Dorval	Satisfaction? 6
Magog	Satisfaction? 9
Berthierville	Satisfaction? 9


  5%|▌         | 1/20 [00:42<13:26, 42.43s/it]



 météo: Blue sky, a 24.461895998263646°C	, temps de traffic: 33.51046841538154min


...

Ahuntsic	Satisfaction? 5
VanHorne	Satisfaction? 2
Verdun	Satisfaction? 2
Lasalle	Satisfaction? 5
Préfontaine	Satisfaction? 3
Boucherville	Satisfaction? 3
Taz	Satisfaction? 1
Spin	Satisfaction? 1
Saint Jérome	Satisfaction? 8
Saint Sauveur	Satisfaction? 9
Assomption	Satisfaction? 8
Benny	Satisfaction? 6
Dorval	Satisfaction? 6
Magog	Satisfaction? 9
Berthierville	Satisfaction? 9


 95%|█████████▌| 19/20 [12:18<00:42, 42.93s/it]



 météo: Blue sky, a 29.6444422676647°C	, temps de traffic: 7.500504979617617min


Ahuntsic	Satisfaction? 6
VanHorne	Satisfaction? 2
Verdun	Satisfaction? 2
Lasalle	Satisfaction? 5
Préfontaine	Satisfaction? 5
Boucherville	Satisfaction? 5
Taz	Satisfaction? 1
Spin	Satisfaction? 1
Saint Jérome	Satisfaction? 8
Saint Sauveur	Satisfaction? 9
Assomption	Satisfaction? 8
Benny	Satisfaction? 6
Dorval	Satisfaction? 5
Magog	Satisfaction? 9
Berthierville	Satisfaction? 9


100%|██████████| 20/20 [12:59<00:00, 38.95s/it]


In [29]:

df = pd.DataFrame(data)
print(df.head(10))

df.to_csv("data/skatepark_dataset_simplified.csv", index=False)
print("\nDone")    

       spot_name  index spot  weather  temperature  traffic_time  satisfaction
0       Ahuntsic           0        2         28.6          23.7           6.0
1       VanHorne           1        2         28.6          23.7           5.0
2         Verdun           2        2         28.6          23.7           3.0
3        Lasalle           3        2         28.6          23.7           6.0
4    Préfontaine           4        2         28.6          23.7           5.0
5   Boucherville           5        2         28.6          23.7           5.0
6            Taz           6        2         28.6          23.7           1.0
7           Spin           7        2         28.6          23.7           1.0
8   Saint Jérome           8        2         28.6          23.7           7.0
9  Saint Sauveur           9        2         28.6          23.7           8.0

Done


## AWS like project:

We will use minIo, to use a local bucket like AWS S3. The goal is to be able to switch on AWS if in the future the project become more serious.
MinIO run In another Docker container, to have access to the GUI:

http://127.0.0.1:9001/login


In [30]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")

In [31]:
s3 = boto3.client(
    "s3",
    endpoint_url = S3_ENDPOINT,
    aws_access_key_id = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
    region_name="us-east-1"    
)

BUCKET_NAME = "skatedata"
FILE_PATH = "data/skatepark_dataset_simplified.csv"
#FILE_PATH = "data/skatepark_dataset_test.csv"
OBJECT_NAME = "cleaned_skatepark_dataset.csv"

try:
    s3.create_bucket(Bucket = BUCKET_NAME)
    print(f"[+] {BUCKET_NAME} bucket created!")
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f"[i] Bucket '{BUCKET_NAME}' already exists...")
    else:
        print(f"[-]{e}")

[i] Bucket 'skatedata' already exists...


In [34]:
s3.upload_file(FILE_PATH, BUCKET_NAME, OBJECT_NAME)

Check if the csv is inside the bucket

In [35]:
print("[i] file inside the bucket:")
for obj in s3.list_objects_v2(Bucket=BUCKET_NAME).get("Contents", []):
    print(f" - {obj['Key']}")

[i] file inside the bucket:
 - cleaned_skatepark_dataset.csv
 - skatepark_dataset_test.csv
